# Testing Notebook 02:

In [1]:
from typing import List
import pandas as pd 
import numpy as np

from statdepth.depth import BandDepth

In [2]:
!where python

/Users/julian/opt/miniconda3/bin/python
/usr/bin/python


Generate a toy DataFrame for testing

In [3]:
df = pd.DataFrame(np.random.randint(0,5,size=(5, 5)), columns=list('ABCDE'))
df

,A,B,C,D,E
0,3,3,1,0,4
1,2,0,0,3,3
2,4,0,1,4,0
3,1,3,3,4,4
4,0,2,2,3,1


In [14]:
bd = BandDepth([df], J=3, relax=True)

bd.sorted()

AttributeError: '_BandDepthSeries' object has no attribute 'sorted'

In [17]:
from typing import get_type_hints, Dict, Union, List

def dummy(l: int, s: str, j: Union[List[str], int]) -> bool:
    return False

hints = get_type_hints(dummy)


In [21]:
hints

{'l': int, 's': str, 'j': typing.Union[typing.List[str], int], 'return': bool}

In [9]:
pd.DataFrame(df)

,0
A,0.18
B,0.34
C,0.32
D,0.50
E,0.12
